# SMTP Test Notebook

This notebook helps you test SMTP credentials and sending a simple email.

WARNING: Do NOT commit real credentials to source control. Use environment variables or VS Code secrets.


# 1) Install required packages (run in terminal if needed)

# If you need python-dotenv for loading a .env file, install it once in your environment:
# !pip install python-dotenv

# smtplib is part of the Python standard library and requires no install.


# 2) Configure Gmail / GoDaddy account

- If your mailbox is Google Workspace (Gmail), enable 2‑Step Verification and create an App Password for "Mail". Use that App Password as `SMTP_PASSWORD`.
- If your mailbox is provisioned by GoDaddy and not Gmail, use GoDaddy's SMTP host (e.g., `smtpout.secureserver.net`) and the password provided by GoDaddy.
- Do NOT paste credentials into public repos. Use environment variables or VS Code secrets.


In [1]:
# 3) Set credentials (HARD-CODE or replace placeholders)

# Replace the placeholders below with the values you want to test. If you prefer not to hardcode them here,
# set them in a .env file and use python-dotenv as shown in the next cell.

SMTP_SERVER = "smtp.gmail.com"   # or your SMTP host (e.g. smtpout.secureserver.net)
SMTP_PORT = 587
SMTP_USER = "Accounts@tigerleads.ai"     # <-- REPLACE with your email (e.g. Accounts@tigerleads.ai)
SMTP_PASS = "Adminaccounts123"         # <-- REPLACE with your password or App Password
EMAIL_FROM = SMTP_USER
TO_EMAIL = "ayeshashahid771771@gmail.com"      # Set a destination email to receive the test


In [2]:
# 5) Send a test email using STARTTLS (port 587)
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


def send_test_email_starttls(smtp_server, smtp_port, smtp_user, smtp_pass, email_from, to_email):
    msg = MIMEMultipart()
    msg['From'] = email_from
    msg['To'] = to_email
    msg['Subject'] = 'SMTP Test Email from notebook'
    body = 'This is a test email sent from smtp_test_notebook.ipynb'
    msg.attach(MIMEText(body, 'plain'))

    context = ssl.create_default_context()
    try:
        with smtplib.SMTP(smtp_server, smtp_port, timeout=30) as server:
            server.ehlo()
            server.starttls(context=context)
            server.ehlo()
            print('Attempting login...')
            server.login(smtp_user, smtp_pass)
            print('Login successful, sending message...')
            server.sendmail(email_from, to_email, msg.as_string())
            print('Email sent successfully')
            return True, None
    except smtplib.SMTPAuthenticationError as e:
        print('Authentication failed:', e)
        return False, e
    except Exception as e:
        print('Failed to send email:', repr(e))
        return False, e


In [ ]:
# 5) Send a test email using SSL (GoDaddy)
def send_test_email_ssl(smtp_server, smtp_port, smtp_user, smtp_pass, email_from, to_email):
    msg = MIMEMultipart()
    msg['From'] = email_from
    msg['To'] = to_email
    msg['Subject'] = 'SMTP Test Email (SSL)'
    body = 'This is a test email sent from smtp_test_notebook using SSL (GoDaddy-style).'
    msg.attach(MIMEText(body, 'plain'))

    context = ssl.create_default_context()
    try:
        with smtplib.SMTP_SSL(smtp_server, smtp_port, context=context, timeout=30) as server:
            server.ehlo()
            print('Attempting login...')
            server.login(smtp_user, smtp_pass)
            print('Login successful, sending message...')
            server.sendmail(email_from, to_email, msg.as_string())
            print('Email sent successfully')
            return True, None
    except smtplib.SMTPAuthenticationError as e:
        print('Authentication failed:', e)
        return False, e
    except Exception as e:
        print('Failed to send email:', repr(e))
        return False, e

# Use GoDaddy SMTP host and SSL port (won't overwrite existing notebook variables)
gd_server = "smtpout.secureserver.net"
gd_port = 465

result_gd, err_gd = send_test_email_ssl(gd_server, gd_port, SMTP_USER, SMTP_PASS, EMAIL_FROM, TO_EMAIL)
print('Result:', result_gd)
if err_gd:
    print('Error details:', repr(err_gd))

In [ ]:
# 6) Run the test (call this cell after you replace the placeholders above)

result, err = send_test_email_starttls(SMTP_SERVER, SMTP_PORT, SMTP_USER, SMTP_PASS, EMAIL_FROM, TO_EMAIL)
print('Result:', result)
if err:
    print('Error details:', repr(err))

# If you see an SMTPAuthenticationError with 535/5.7.8 BadCredentials, follow the App Password or provider steps in the notes above.


Attempting login...
Authentication failed: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials ffacd0b85a97d-42b2fb308f5sm24343043f8f.42 - gsmtp')
Result: False
Error details: SMTPAuthenticationError(535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials ffacd0b85a97d-42b2fb308f5sm24343043f8f.42 - gsmtp')


: 

In [1]:
import smtplib, ssl

smtp_server = "smtp.gmail.com"
port = 465  # SSL port
email = "Accounts@tigerleads.ai"
password = "Adminaccounts123"  # Use app password if 2FA enabled

context = ssl.create_default_context()
try:
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(email, password)
        print("SMTP login successful!")
except smtplib.SMTPAuthenticationError:
    print("Login failed: wrong username/password or 2FA required")
except Exception as e:
    print("Error connecting to SMTP server:", e)


Login failed: wrong username/password or 2FA required


# Troubleshooting notes

- If you get (535, '5.7.8 Username and Password not accepted') -> create an App Password for Google Workspace or use the provider's correct SMTP host and credentials.
- If logo or attachments cause errors, simplify the message first and test with plain text.
- For production, use a transactional email provider (SendGrid/Mailgun/Postmark/AWS SES) or OAuth2 for Gmail.
- After testing, revoke any App Passwords you created and remove credentials from your notebook and repo.
